In [20]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import QuantileTransformer
from sklearn.model_selection import GridSearchCV

In [21]:
def get_score(true_values, predicted_values):
    mae_ = mean_absolute_error(true_values, predicted_values)
    mse_ = mean_squared_error(true_values, predicted_values)
    r2_ = r2_score(true_values, predicted_values)

    return mae_, mse_, r2_

# Loading data

### Using only values for FaceAPI without neutral gave the best result

In [22]:
dataset = pd.read_csv('dataset.csv')

In [23]:
dataset = dataset.loc[dataset['COND'] != '0']

data = dataset.iloc[:, :8]
labels = dataset.iloc[:, -2:]

In [24]:
data_train_full, data_test, labels_train_full, labels_test = train_test_split(data, labels, test_size=0.2, random_state=123)

data_train, data_validation, labels_train, labels_validation = train_test_split(data_train_full, labels_train_full, test_size=0.2, random_state=123)

data_train = data_train.loc[labels_train['ANS_AROUSAL'] != 1]
data_train = data_train.loc[labels_train['ANS_AROUSAL'] != 5]
data_train = data_train.loc[labels_train['ANS_AROUSAL'] != 9]
labels_train = labels_train.loc[labels_train['ANS_AROUSAL'] != 1]
labels_train = labels_train.loc[labels_train['ANS_AROUSAL'] != 5]
labels_train = labels_train.loc[labels_train['ANS_AROUSAL'] != 9]

data_train = data_train.loc[labels_train['ANS_VALENCE'] != 1]
data_train = data_train.loc[labels_train['ANS_VALENCE'] != 5]
data_train = data_train.loc[labels_train['ANS_VALENCE'] != 9]
labels_train = labels_train.loc[labels_train['ANS_VALENCE'] != 1]
labels_train = labels_train.loc[labels_train['ANS_VALENCE'] != 5]
labels_train = labels_train.loc[labels_train['ANS_VALENCE'] != 9]

# Scaling input values to Gaussian distributions

In [25]:
quantile_transformer = QuantileTransformer(output_distribution='normal', random_state=0)
data_train_qt = quantile_transformer.fit_transform(data_train)
data_train_qt = pd.DataFrame(data_train_qt, columns=data_train.columns)
data_validation_qt = quantile_transformer.fit_transform(data_validation)
data_validation_qt = pd.DataFrame(data_validation_qt, columns=data_validation.columns)

# Training

In [26]:
etr = ExtraTreesRegressor(random_state=0)

etr.fit(data_train_qt, labels_train.iloc[:, 1])
print('Training finished')

Training finished


In [27]:
predictions_etr = etr.predict(data_validation_qt)

mae_etr, mse_etr, r2_etr = get_score(labels_validation.iloc[:, 1], predictions_etr)
print(f'''Values for validation set:\nMAE: {mae_etr}\nMSE: {mse_etr}\nRMSE: {mse_etr**.5}\nR2:  {r2_etr}''')

Values for validation set:
MAE: 1.6626769500796903
MSE: 4.382582844745869
RMSE: 2.093461928181611
R2:  -0.10167736274204797


# Optimizing parameters

In [28]:
# param_grid = [{
#     'n_estimators': [100, 200, 300, 400, 500],
#     'min_samples_split': [2, 4, 8, 16, 32],
#     'min_samples_leaf': [1, 2, 4, 8, 16]
# }]

In [29]:
# clf = GridSearchCV(estimator=etr, param_grid=param_grid, scoring='neg_root_mean_squared_error', n_jobs=2, cv=3, verbose=1)
# clf.fit(data_train, labels_train.iloc[:, 1])
# print('Search finished')

In [30]:
# clf.best_params_

In [31]:
# predictions_clf = clf.predict(data_validation_qt)
#
# mae_clf, mse_clf, r2_clf = get_score(labels_validation.iloc[:, 1], predictions_clf)
# print(f'''Values for validation set:\nMAE: {mae_clf}\nMSE: {mse_clf}\nRMSE: {mse_clf**.5}\nR2:  {r2_clf}''')

### Using best parameters

In [32]:
etr_best = ExtraTreesRegressor(n_estimators=500, min_samples_split=2, min_samples_leaf=16, random_state=0, n_jobs=2)

etr_best.fit(data_train_qt, labels_train.iloc[:, 1])
print('Training finished')

Training finished


In [33]:
predictions_etr_best = etr_best.predict(data_validation_qt)

mae_etr_best, mse_etr_best, r2_etr_best = get_score(labels_validation.iloc[:, 1], predictions_etr_best)
print(f'''Values for validation set:\nMAE: {mae_etr_best}\nMSE: {mse_etr_best}\nRMSE: {mse_etr_best**.5}\nR2:  {r2_etr_best}''')

Values for validation set:
MAE: 1.582606013803311
MSE: 3.981575509127299
RMSE: 1.995388560939272
R2:  -0.0008736313364643422


# Using full dataset

In [34]:
dataset = pd.read_csv('dataset_full.csv')
dataset = dataset.loc[dataset['COND'] != '0']

data = dataset.iloc[:, :8]
labels = dataset.iloc[:, -2:]
data_train_full, data_test, labels_train_full, labels_test = train_test_split(data, labels, test_size=0.2,
                                                                              random_state=123)

data_train, data_validation, labels_train, labels_validation = train_test_split(data_train_full, labels_train_full,
                                                                                test_size=0.2, random_state=123)

data_train = data_train.loc[labels_train['ANS_AROUSAL'] != 1]
data_train = data_train.loc[labels_train['ANS_AROUSAL'] != 5]
data_train = data_train.loc[labels_train['ANS_AROUSAL'] != 9]
labels_train = labels_train.loc[labels_train['ANS_AROUSAL'] != 1]
labels_train = labels_train.loc[labels_train['ANS_AROUSAL'] != 5]
labels_train = labels_train.loc[labels_train['ANS_AROUSAL'] != 9]

data_train = data_train.loc[labels_train['ANS_VALENCE'] != 1]
data_train = data_train.loc[labels_train['ANS_VALENCE'] != 5]
data_train = data_train.loc[labels_train['ANS_VALENCE'] != 9]
labels_train = labels_train.loc[labels_train['ANS_VALENCE'] != 1]
labels_train = labels_train.loc[labels_train['ANS_VALENCE'] != 5]
labels_train = labels_train.loc[labels_train['ANS_VALENCE'] != 9]

In [35]:
quantile_transformer = QuantileTransformer(output_distribution='normal', random_state=0)
data_train_qt = quantile_transformer.fit_transform(data_train)
data_train_qt = pd.DataFrame(data_train_qt, columns=data_train.columns)
data_validation_qt = quantile_transformer.fit_transform(data_validation)
data_validation_qt = pd.DataFrame(data_validation_qt, columns=data_validation.columns)

In [36]:
etr_best = ExtraTreesRegressor(n_estimators=500, min_samples_split=2, min_samples_leaf=16, random_state=0, n_jobs=2)

etr_best.fit(data_train_qt, labels_train.iloc[:, 1])
print('Training finished')

Training finished


In [37]:
predictions_etr_best = etr_best.predict(data_validation_qt)

mae_etr_best, mse_etr_best, r2_etr_best = get_score(labels_validation.iloc[:, 1], predictions_etr_best)
print(f'''Values for validation set:\nMAE: {mae_etr_best}\nMSE: {mse_etr_best}\nRMSE: {mse_etr_best**.5}\nR2:  {r2_etr_best}''')

Values for validation set:
MAE: 1.608056934865649
MSE: 4.079171688243261
RMSE: 2.019695939552105
R2:  -0.002189561824205599
